# Klasifikácia rozhodovacím stromom

In [1]:
from sklearn import tree
import pandas as pd
import numpy as np
import random as rnd
import seaborn
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
import graphviz 

pd.set_option('display.max_columns', 50)

In [2]:
train_data = pd.read_csv('data/train_predspracovane.csv')
test_data = pd.read_csv('data/test_predspracovane.csv')

Do funkcie pre predspracovanie sme zabudli vložiť riadok kódu, kde sa vymazával stĺpec date_of_birth. Urobíme teda túto poslednú úpravu teraz.

In [3]:
train_data = train_data.drop('id', axis=1)
test_data = test_data.drop('id', axis=1)

Rozvinuli sme hodnoty kategorických atribútov do vlastných stĺpcov podľa One Hot Encoding, takže rozhodovací strom bude môcť pracovať aj s nimi.

In [4]:
tree_train_data = train_data.drop('test', axis=1)
tree_train_data = pd.get_dummies(tree_train_data, columns=tree_train_data.columns[tree_train_data.dtypes == np.object], dummy_na=True)
tree_train_target = train_data.test.copy()

tree_test_data = test_data.drop('test', axis=1)
tree_test_data = pd.get_dummies(tree_test_data, columns=tree_test_data.columns[tree_test_data.dtypes == np.object], dummy_na=True)
tree_test_target = test_data.test.copy()

Transformovali sme klasifikovaný atribút na binárny, aby sme mohli pre optimalizáciu rozhodovacieho stromu použiť metriku f1.

In [5]:
def bintransform(target):
    ret = []
    for i in range(len(target)):
        if (target.iloc[i] == 'discordant'):
            ret.append(1)
        else:
            ret.append(0)
    return ret
            
tree_train_target = bintransform(tree_train_target)
tree_test_target = bintransform(tree_test_target)

Zahodili sme stĺpce vyjadrujúce tie hodnoty kategorických atribútov, ktoré sa nevyskytovali v trénovacom dátovom súbore, takže na základe nich nebolo možné klasifikovať a boli zbytočné.

In [6]:
toDrop = []
for attr in tree_test_data.columns:
    if attr not in tree_train_data.columns:
        toDrop.append(attr)

tree_test_data = tree_test_data.drop(toDrop, axis=1)

A do testovacích dát sme pre zmenu doplnili stĺpce vyjadrujúce tie hodnoty kategorických atribútov, ktoré sa nevyskytovali v testovacom dátovom súbore.

In [7]:
for attr in tree_train_data.columns:
    if attr not in tree_test_data.columns:
        tree_test_data[attr] = 0

# 1 Hľadanie optimálneho stromu

Vytvorili sme rozhodovací strom a pokúsili sa nájsť optimálne hodnoty hyperparametrov, takže môžeme náš strom optimalizovať najviac ako to je možné.

In [8]:
cv_params={'criterion': ['gini', 'entropy'], 'min_samples_leaf':range(1, 10), 'min_samples_split':[0.000001, 0.1, 0.2, 0.3], 'max_depth':list(range(1, train_data.shape[1])), 'class_weight':["balanced", None]}   

tree_classifier = DecisionTreeClassifier
ind_params = {'random_state': 0}
optimization = GridSearchCV(tree_classifier(**ind_params), 
                            cv_params, 
                             scoring = 'f1', cv = 10, n_jobs = -1, verbose=True)

In [9]:
optimization.fit(tree_train_data, tree_train_target)

Fitting 10 folds for each of 5616 candidates, totalling 56160 fits


[Parallel(n_jobs=-1)]: Done  78 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 1278 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done 3278 tasks      | elapsed:   25.7s
[Parallel(n_jobs=-1)]: Done 6078 tasks      | elapsed:   44.8s
[Parallel(n_jobs=-1)]: Done 9678 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 14078 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 19278 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 25278 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 32078 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 39678 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 48078 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 56160 out of 56160 | elapsed:  7.1min finished


GridSearchCV(cv=10, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'criterion': ['gini', 'entropy'], 'min_samples_leaf': range(1, 10), 'min_samples_split': [1e-06, 0.1, 0.2, 0.3], 'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39], 'class_weight': ['balanced', None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=True)

Zistili sme najlepšiu kombináciu hyperparametrov pre naše dáta. Tieto použijeme pri trénovaní nášho rozhodovacieho stromu.

In [10]:
sorted(optimization.grid_scores_, key=lambda x: x.mean_validation_score, reverse=True)[:5]

C:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.72377, std: 0.13889, params: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 7, 'min_samples_leaf': 3, 'min_samples_split': 1e-06},
 mean: 0.70606, std: 0.13487, params: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 3, 'min_samples_split': 1e-06},
 mean: 0.69925, std: 0.12488, params: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 9, 'min_samples_leaf': 3, 'min_samples_split': 1e-06},
 mean: 0.69925, std: 0.12488, params: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 3, 'min_samples_split': 1e-06},
 mean: 0.69925, std: 0.12488, params: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 11, 'min_samples_leaf': 3, 'min_samples_split': 1e-06}]

Natrénovali sme si strom podľa zistenej najoptimálnejšej kombinácie hyperparametrov.

In [11]:
tree_classifier = DecisionTreeClassifier(criterion='entropy', max_depth=7, class_weight=None, min_samples_leaf= 3, min_samples_split= 1e-06)
tree_classifier = tree_classifier.fit(tree_train_data, tree_train_target)

Strom sme si nechali vizualizovať, aby sme zistili, na akých pravidlách je založený a či tieto pravidlá nejak súvisia s našimi zisteniami v časti exploratívnej analýzy.

In [12]:
tree.export_graphviz(tree_classifier, feature_names=tree_train_data.columns, filled=True, class_names='test',  out_file='strom_e7n3.dot')

<img src="strom_e7n3.png">

Vytvorili sme si funkciu, ktorá pre nás vytvorí pravdivostnú tabuľku pre výsledky jednotlivých rozhodovacích stromov, ktorá nám pomôže vyhodnotiť ich výsledky.

In [13]:
def pravdivost(real, pred, otocena=False):
    com = pd.DataFrame()
    com['real'] = real
    com['pred'] = pred
    
    total = len(real)
    tp = 0
    tIe = 0
    tIIe = 0
    tn = 0
    if  not otocena:
        for i in range(len(real)):
            if (pred[i] == 1 and real[i] == 1):
                tp = tp + 1
            if (pred[i] == 1 and real[i] == 0):
                tIe = tIe + 1
            if (pred[i] == 0 and real[i] == 1):
                tIIe = tIIe + 1
            if (pred[i] == 0 and real[i] == 0):
                tn = tn + 1
    else:
        for i in range(len(real)):
            if (pred[i] == 0 and real[i] == 0):
                tp = tp + 1
            if (pred[i] == 0 and real[i] == 1):
                tIe = tIe + 1
            if (pred[i] == 1 and real[i] == 0):
                tIIe = tIIe + 1
            if (pred[i] == 1 and real[i] == 1):
                tn = tn + 1
        
    print(64*'═')
    print('║{:<20}║{:<20}|{:<20}║'.format('', 'Prediction positive', 'Prediction negative'))
    print(64*'═')
    print('║{:<20}║{:<20.2f}|{:<20.2f}║'.format('Condition positive', tp, tIIe))
    print(64*'═')
    print('║{:<20}║{:<20.2f}|{:<20.2f}║'.format('Condition negative', tIe, tn))
    print(64*'═')
        
    print("Accuracy: ", (tp + tn) / total if total != 0 else 0)
    print("Precision: ",  tp / (tp + tIe) if tp + tIe != 0 else 0)
    print("Recall: ", tp / (tp + tIIe) if tp + tIIe != 0 else 0)

Overili sme si presnosť klasifikátora pre predikovanie pozitívnych testov.

Zistili sme, že výsledky pre predikovanie negatínej triedy sú síce dobré, ale pre predikovanie pozitívneho veľmi zlé.Podľa tabuľky je možné pozorovať, že omnoho častejšie sa klasifikátor pri predikovaní pozitívneho testu mýli, ako má pravdu. Jeho použíteľnosť pre predikovanie pozitívneho testu v testovacom súbore je teda veľmi zlá.

In [14]:
tree_prediction = tree_classifier.predict(tree_train_data)
pravdivost(tree_train_target, tree_prediction)

════════════════════════════════════════════════════════════════
║                    ║Prediction positive |Prediction negative ║
════════════════════════════════════════════════════════════════
║Condition positive  ║39.00               |6.00                ║
════════════════════════════════════════════════════════════════
║Condition negative  ║5.00                |2750.00             ║
════════════════════════════════════════════════════════════════
Accuracy:  0.9960714285714286
Precision:  0.8863636363636364
Recall:  0.8666666666666667


In [15]:
tree_prediction = tree_classifier.predict(tree_test_data)
pravdivost(tree_test_target, tree_prediction)

════════════════════════════════════════════════════════════════
║                    ║Prediction positive |Prediction negative ║
════════════════════════════════════════════════════════════════
║Condition positive  ║1.00                |12.00               ║
════════════════════════════════════════════════════════════════
║Condition negative  ║16.00               |943.00              ║
════════════════════════════════════════════════════════════════
Accuracy:  0.9711934156378601
Precision:  0.058823529411764705
Recall:  0.07692307692307693


Pre nás je zaujímavé hlavne správne predikovanie pozitívnych testov. Tieto značia, že subjekt môže mať danú chorobu a preto je pre nás dôležitejšie vedieť identifikovať čo najviac takýchto prípadov ako presnejšie predikovanie zdravých ľudí. Ak je zdravý človek mylne označovaný za chorého, nič závažného sa nestane. Ak je ale chorý považovaný za zdravého, môže ho to stáť život.

Klasifikátor v aktuálnom stave nedosahuje z tohto hľadiska dobré výsledky. Pokúsime sa teda ešte upraviť ručne niektoré hyperparametre a natrénovať strom znova.

Vyskúšame nastaviť vyvažovanie tried pri klasifikácii, keďže v našom súbore sú triedy nevyvážené.

In [16]:
print(len(train_data[train_data.test == 'negative']), len(train_data[train_data.test == 'discordant']))

2755 45


Klasifikácia sa mierne zlepšila z pohľadu správne predikovaných pozitívnych tried. Značí to teda, že sa najskôr GridSearchCV nesanží optimalizovať na také výsledky, aké by sme chceli my.

In [17]:
tree_classifier = DecisionTreeClassifier(criterion='entropy', max_depth=7, class_weight='balanced', min_samples_leaf= 3, min_samples_split= 1e-06)
tree_classifier = tree_classifier.fit(tree_train_data, tree_train_target)
tree_prediction = tree_classifier.predict(tree_train_data)
pravdivost(tree_train_target, tree_prediction)
tree_prediction = tree_classifier.predict(tree_test_data)
pravdivost(tree_test_target, tree_prediction)

════════════════════════════════════════════════════════════════
║                    ║Prediction positive |Prediction negative ║
════════════════════════════════════════════════════════════════
║Condition positive  ║45.00               |0.00                ║
════════════════════════════════════════════════════════════════
║Condition negative  ║23.00               |2732.00             ║
════════════════════════════════════════════════════════════════
Accuracy:  0.9917857142857143
Precision:  0.6617647058823529
Recall:  1.0
════════════════════════════════════════════════════════════════
║                    ║Prediction positive |Prediction negative ║
════════════════════════════════════════════════════════════════
║Condition positive  ║3.00                |10.00               ║
════════════════════════════════════════════════════════════════
║Condition negative  ║37.00               |922.00              ║
════════════════════════════════════════════════════════════════
Accuracy:  0.951

Vyskúšame teda použiť optimalizovať na metriku recall, ktorá by mala znižovať chybu 2. stupňa, ktorej sa chceme vyvarovať.

In [18]:
cv_params={'criterion': ['gini', 'entropy'], 'min_samples_leaf':range(1, 10), 'min_samples_split':[0.000001, 0.1, 0.2, 0.3], 'max_depth':list(range(1, train_data.shape[1])), 'class_weight':["balanced", None]}            

tree_classifier_recall = DecisionTreeClassifier
ind_params = {'random_state': 0}
optimization = GridSearchCV(tree_classifier_recall(**ind_params), 
                            cv_params, 
                             scoring = 'recall', cv = 10, n_jobs = -1, verbose=True)

In [19]:
optimization.fit(tree_train_data, tree_train_target)

Fitting 10 folds for each of 5616 candidates, totalling 56160 fits


[Parallel(n_jobs=-1)]: Done  50 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 1850 tasks      | elapsed:   19.7s
[Parallel(n_jobs=-1)]: Done 4850 tasks      | elapsed:   42.1s
[Parallel(n_jobs=-1)]: Done 9050 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 14450 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 21050 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 28850 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 37850 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 48050 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 56160 out of 56160 | elapsed:  6.9min finished


GridSearchCV(cv=10, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'criterion': ['gini', 'entropy'], 'min_samples_leaf': range(1, 10), 'min_samples_split': [1e-06, 0.1, 0.2, 0.3], 'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39], 'class_weight': ['balanced', None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='recall', verbose=True)

In [20]:
sorted(optimization.grid_scores_, key=lambda x: x.mean_validation_score, reverse=True)[:5]

C:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.93011, std: 0.15524, params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 1, 'min_samples_split': 0.1},
 mean: 0.93011, std: 0.15524, params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 1, 'min_samples_split': 0.2},
 mean: 0.93011, std: 0.15524, params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 1, 'min_samples_split': 0.3},
 mean: 0.93011, std: 0.15524, params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2, 'min_samples_split': 0.1},
 mean: 0.93011, std: 0.15524, params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2, 'min_samples_split': 0.2}]

In [21]:
tree_classifier_recall = DecisionTreeClassifier(criterion='gini', max_depth=2, class_weight='balanced', min_samples_leaf= 1, min_samples_split= 0.1)
tree_classifier_recall = tree_classifier_recall.fit(tree_train_data, tree_train_target)

Pozreli sme sa, ako sa zmenili rozhodovacie pravidlá.

In [23]:
tree.export_graphviz(tree_classifier_recall, feature_names=tree_train_data.columns, filled=True, class_names='test',  out_file='strom_g2b1.dot')

<img src="strom_g2b1.png">

Výsledky na nám podarilo výrazne zlepšiť. Pri trénovacom súbore už nebola dosiahnutá žiadna chyba 2.stupňa, čo sme si želali. Pri testovacom súbore výsledky síce nie sú tie najoptimálnejšie, ale aj tu už je badateľné zlepšenie, ktoré robí strom aspoň použiteľným.

In [24]:
tree_recall_prediction = tree_classifier.predict(tree_train_data)
pravdivost(tree_train_target, tree_recall_prediction)

════════════════════════════════════════════════════════════════
║                    ║Prediction positive |Prediction negative ║
════════════════════════════════════════════════════════════════
║Condition positive  ║45.00               |0.00                ║
════════════════════════════════════════════════════════════════
║Condition negative  ║23.00               |2732.00             ║
════════════════════════════════════════════════════════════════
Accuracy:  0.9917857142857143
Precision:  0.6617647058823529
Recall:  1.0


S ňou sme ale lepšie výsledky nezískali.

In [25]:
tree_recall_prediction = tree_classifier_recall.predict(tree_test_data)
pravdivost(tree_test_target, tree_recall_prediction)

════════════════════════════════════════════════════════════════
║                    ║Prediction positive |Prediction negative ║
════════════════════════════════════════════════════════════════
║Condition positive  ║8.00                |5.00                ║
════════════════════════════════════════════════════════════════
║Condition negative  ║75.00               |884.00              ║
════════════════════════════════════════════════════════════════
Accuracy:  0.9176954732510288
Precision:  0.0963855421686747
Recall:  0.6153846153846154


Ako posledný pokus o zlepšenie výsledkov pre pozitívnu predikciu sa pokúsime využiť pozorovanie, že pri niektorých stĺpoch je pri pozitívnom teste menšia odchýľka ich hodnôt od priemeru stĺpca ako pri negatívnom teste.

In [26]:
def addDeviation(data):
    aug = data.copy()
    for col in ['TSH', 'T3', 'TT4', 'T4U', 'fnlwgt', 'FTI']:
        mn = data[col].mean()
        new = col + '_deviation'
        aug[new] = 0 
        for i in range(len(data[col])):
            aug.loc[i, new] = abs(data[col][i] - mn)
    return aug

Upravíme teda dátové súbory.

In [27]:
dev_train_data = addDeviation(tree_train_data)
dev_test_data = addDeviation(tree_test_data)

In [28]:
dev_train_data.head()

,age,TSH,T3,TT4,T4U,fnlwgt,education-num,hours-per-week,FTI,birth_year,birth_month,on thyroxine_f,on thyroxine_t,on thyroxine_nan,query on thyroxine_f,query on thyroxine_t,query on thyroxine_nan,pregnant_f,pregnant_t,pregnant_nan,thyroid surgery_f,thyroid surgery_t,thyroid surgery_nan,I131 treatment_f,I131 treatment_t,...,on antithyroid medication_f,on antithyroid medication_t,on antithyroid medication_nan,sick_f,sick_t,sick_nan,T4U measured_f,T4U measured_t,T4U measured_nan,psych_f,psych_t,psych_nan,query hypothyroid_f,query hypothyroid_t,query hypothyroid_nan,yield_gain,yield_loss,yield_none,yield_nan,TSH_deviation,T3_deviation,TT4_deviation,T4U_deviation,fnlwgt_deviation,FTI_deviation
0,82.0,0.813772,1.0,68.0,0.77,365.447207,11.0,40.0,88.0,1935.0,5.0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,...,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0.667723,1.024966,41.132816,0.225006,29.803774,22.429851
1,50.0,5.172593,0.5,22.0,1.12,402.661130,13.0,35.0,19.0,1967.0,8.0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,...,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,5.026544,1.524966,87.132816,0.124994,7.410149,91.429851
2,41.0,0.341027,2.0,99.0,0.96,318.259586,10.0,50.0,104.0,1976.0,5.0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,...,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0.194978,0.024966,10.132816,0.035006,76.991395,6.429851
3,73.0,-0.351650,2.0,116.0,0.99,483.595580,9.0,25.0,117.0,1944.0,10.0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,...,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0.497699,0.024966,6.867184,0.005006,88.344599,6.570149
4,64.0,0.712661,1.8,80.0,0.91,550.637942,10.0,60.0,88.0,1953.0,5.0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,...,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0.566613,0.224966,29.132816,0.085006,155.386960,22.429851


Použijeme optimalizáciu pre recall.

In [31]:
cv_params={'criterion': ['gini', 'entropy'], 'min_samples_leaf':range(1, 10), 'min_samples_split':[0.000001, 0.1, 0.2, 0.3], 'max_depth':list(range(1, train_data.shape[1])), 'class_weight':["balanced", None]}            

tree_classifier_dev = DecisionTreeClassifier
ind_params = {'random_state': 0}
optimization = GridSearchCV(tree_classifier_dev(**ind_params), 
                            cv_params, 
                             scoring = 'recall', cv = 10, n_jobs = -1, verbose=True)

In [32]:
optimization.fit(dev_train_data, tree_train_target)

Fitting 10 folds for each of 5616 candidates, totalling 56160 fits


[Parallel(n_jobs=-1)]: Done  85 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 1285 tasks      | elapsed:   11.7s
[Parallel(n_jobs=-1)]: Done 3285 tasks      | elapsed:   28.0s
[Parallel(n_jobs=-1)]: Done 6085 tasks      | elapsed:   51.6s
[Parallel(n_jobs=-1)]: Done 9685 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 14085 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 19285 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 25285 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 32085 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 39685 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 48085 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done 56160 out of 56160 | elapsed:  7.6min finished


GridSearchCV(cv=10, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'criterion': ['gini', 'entropy'], 'min_samples_leaf': range(1, 10), 'min_samples_split': [1e-06, 0.1, 0.2, 0.3], 'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39], 'class_weight': ['balanced', None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='recall', verbose=True)

In [33]:
sorted(optimization.grid_scores_, key=lambda x: x.mean_validation_score, reverse=True)[:5]

C:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.93011, std: 0.15524, params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 1, 'min_samples_split': 0.1},
 mean: 0.93011, std: 0.15524, params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 1, 'min_samples_split': 0.2},
 mean: 0.93011, std: 0.15524, params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 1, 'min_samples_split': 0.3},
 mean: 0.93011, std: 0.15524, params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2, 'min_samples_split': 0.1},
 mean: 0.93011, std: 0.15524, params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2, 'min_samples_split': 0.2}]

In [34]:
tree_classifier_dev = DecisionTreeClassifier(criterion='gini', max_depth=2, class_weight='balanced', min_samples_leaf= 1, min_samples_split= 0.1)
tree_classifier_dev = tree_classifier_dev.fit(dev_train_data, tree_train_target)

Pozreli sme sa, či sa zmenili rozhodovacie pravidlá.

In [36]:
tree.export_graphviz(tree_classifier_recall, feature_names=tree_train_data.columns, filled=True, class_names='test',  out_file='strom_dev_g2b1.dot')

<img src="strom_dev_g2b1.png">

Takáto úprava ale už viac nepomohla.

In [37]:
pravdivost(tree_test_target, tree_classifier_dev.predict(dev_test_data))

════════════════════════════════════════════════════════════════
║                    ║Prediction positive |Prediction negative ║
════════════════════════════════════════════════════════════════
║Condition positive  ║8.00                |5.00                ║
════════════════════════════════════════════════════════════════
║Condition negative  ║75.00               |884.00              ║
════════════════════════════════════════════════════════════════
Accuracy:  0.9176954732510288
Precision:  0.0963855421686747
Recall:  0.6153846153846154


Testovali sme ďalej rôzne rozsahy jednotlivých hyperparametrov, ale nepodarilo sa nám už dosiahnúť taký istý výsledok, ako pri už uvedenej optimaliácii pomocou recall. Zdá sa, že toto je strop pre to, čo môže strom dosiahnúť.

Na základe týchto zistení sme sa teda rozhodli používať model, ktorý sme získli pri optimalizácii recall. Recall ako metrika zo svojej podstaty aj nejlepšie vystihuje, aké výsledky chceme dosiahnúť. 

In [38]:
tree_classifier_final = DecisionTreeClassifier(criterion='gini', max_depth=2, class_weight='balanced', min_samples_leaf= 1, min_samples_split= 0.1)
tree_classifier_final = tree_classifier_final.fit(tree_train_data, tree_train_target)

# 2 Porovnanie metód nahrádzania chýbajúcich hodnôt

Ked už sme sa teda rozhodli, ktorý model by bol najvhodnejšie použíť, rovnaké úpravy sme vykonali aj pre dátové súbory, v ktorých sme neznáme hodnoty nahrádzali buď výlučne priemerom/mediánom, alebo lineárnou regresiou, takže sme ich mohli navzájom porovnať.

In [39]:
mean_med_data = pd.read_csv('data/train_mean_med.csv')
lin_reg_data = pd.read_csv('data/train_linreg.csv')
mean_med_test_data = pd.read_csv('data/test_mean_med_predspracovane.csv')
lin_reg_test_data = pd.read_csv('data/test_linreg_predspracovane.csv')

In [40]:
mean_med_data = mean_med_data.drop('id', axis=1)
lin_reg_data = lin_reg_data.drop('id', axis=1)
mean_med_test_data = mean_med_test_data.drop('id', axis=1)
lin_reg_test_data = lin_reg_test_data.drop('id', axis=1)

In [41]:
mean_med_target = mean_med_data.test
mean_med_data = mean_med_data.drop('test', axis=1)
mean_med_data = pd.get_dummies(mean_med_data, columns=mean_med_data.columns[mean_med_data.dtypes == np.object], dummy_na=True)

lin_reg_target = lin_reg_data.test
lin_reg_data = lin_reg_data.drop('test', axis=1)
lin_reg_data = pd.get_dummies(lin_reg_data, columns=lin_reg_data.columns[lin_reg_data.dtypes == np.object], dummy_na=True)

mean_med_test_target = mean_med_test_data.test
mean_med_test_data = lin_reg_test_data.drop('test', axis=1)
mean_med_test_data = pd.get_dummies(mean_med_test_data, columns=mean_med_test_data.columns[mean_med_test_data.dtypes == np.object], dummy_na=True)

lin_reg_test_target = lin_reg_test_data.test
lin_reg_test_data = lin_reg_test_data.drop('test', axis=1)
lin_reg_test_data = pd.get_dummies(lin_reg_test_data, columns=lin_reg_test_data.columns[lin_reg_test_data.dtypes == np.object], dummy_na=True)

In [42]:
toDrop = []
for attr in mean_med_test_data.columns:
    if attr not in mean_med_data.columns:
        toDrop.append(attr)

mean_med_test_data = mean_med_test_data.drop(toDrop, axis=1)

for attr in mean_med_data.columns:
    if attr not in mean_med_test_data.columns:
        mean_med_test_data[attr] = 0
        
toDrop = []
for attr in lin_reg_test_data.columns:
    if attr not in lin_reg_data.columns:
        toDrop.append(attr)

lin_reg_test_data = lin_reg_test_data.drop(toDrop, axis=1)

for attr in lin_reg_data.columns:
    if attr not in lin_reg_test_data.columns:
        lin_reg_test_data[attr] = 0

In [43]:
mean_med_target = bintransform(mean_med_target)
lin_reg_target = bintransform(lin_reg_target)
mean_med_test_target = bintransform(mean_med_test_target)
lin_reg_test_target = bintransform(lin_reg_test_target)

In [44]:
tree_mean_med_classifier = DecisionTreeClassifier(criterion='gini', max_depth=2, class_weight='balanced', min_samples_leaf= 1, min_samples_split= 0.1)
tree_mean_med_classifier = tree_mean_med_classifier.fit(mean_med_data, mean_med_target)

tree_lin_reg_classifier = DecisionTreeClassifier(criterion='gini', max_depth=2, class_weight='balanced', min_samples_leaf= 1, min_samples_split= 0.1)
tree_lin_reg_classifier = tree_lin_reg_classifier.fit(lin_reg_data, lin_reg_target)

Priemer/medián dosiahol spomedzi troch stromov najhorie výsledky, lineárna regresia bola druhá najhoršie horšia, avšak rozdiel medzi priemrommediánnom a regresiou je o 1 chybu 1. stupňa. Na základe tohto teda môžeme povedať, že náš skorší úsudok o tom ,že kombinácia týchto techník bude lepšia ako každá technika zvlášť bol správny. Dané je to prevdepodobne tým, že každá technika je lepšia a horšia na inej časti dátového súboru a tak hoci sú samostatne obe takmer rovnako zlé, pri správnom skombinovaní a využitím silných stránok každej z nich je možné dosiahnúť lepší výsledok.

Nutno však taktiež poznamenať, že ani jednotlivé tieto techniky sa už výrazne nelíšili od ich kominácie. Rozdiel bol v 15, resp. 16 chybách 1. stupňa.

In [45]:
tree_mean_med_prediction = tree_mean_med_classifier.predict(mean_med_test_data)
pravdivost(mean_med_test_target, tree_mean_med_prediction)

════════════════════════════════════════════════════════════════
║                    ║Prediction positive |Prediction negative ║
════════════════════════════════════════════════════════════════
║Condition positive  ║8.00                |5.00                ║
════════════════════════════════════════════════════════════════
║Condition negative  ║91.00               |868.00              ║
════════════════════════════════════════════════════════════════
Accuracy:  0.9012345679012346
Precision:  0.08080808080808081
Recall:  0.6153846153846154


In [46]:
tree_lin_reg_prediction = tree_lin_reg_classifier.predict(lin_reg_test_data)
pravdivost(lin_reg_test_target, tree_lin_reg_prediction)

════════════════════════════════════════════════════════════════
║                    ║Prediction positive |Prediction negative ║
════════════════════════════════════════════════════════════════
║Condition positive  ║8.00                |5.00                ║
════════════════════════════════════════════════════════════════
║Condition negative  ║90.00               |869.00              ║
════════════════════════════════════════════════════════════════
Accuracy:  0.9022633744855967
Precision:  0.08163265306122448
Recall:  0.6153846153846154


Nechali sme si vizualizovať všetky tri vytvorené stromy (skôr stromčeky), aby sme sa pozreli, aké rozdiely sú v ich rozhodovacích pravidlách.

K žiadným zmenám už ale nedošlo. Odlišná úspešnosť klasifikácie bola očividne založná len na odlišných vstupných dátach.

In [47]:
tree.export_graphviz(tree_classifier_final, feature_names=tree_train_data.columns, filled=True, class_names='test',  out_file='strom.dot')
tree.export_graphviz(tree_mean_med_classifier, feature_names=tree_train_data.columns, filled=True, class_names='test',  out_file='strom_priemer_median.dot') 
tree.export_graphviz(tree_lin_reg_classifier, feature_names=tree_train_data.columns, filled=True, class_names='test',  out_file='strom_linreg.dot') 

### Kominácia
<img src="strom_g2b1.png">

### Priemer-medián
<img src="strom_priemer_median.png">

### Lineárna regresia
<img src="strom_linreg.png">

# 3 Pravidlá

In [48]:
class Pravidlo:
    def __init__(self):
        self.popis = ''
        self.podmienka = lambda a:True
        self.dosledok = lambda a:True
        
    def __init__(self, pop, pod, dos):
        self.popis = pop
        self.podmienka = pod
        self.dosledok = dos
        
    def over(self, data):
        total = data.shape[0]
        tp = data[self.podmienka(data) & self.dosledok(data)].shape[0]
        tIe = data[self.podmienka(data) & ~self.dosledok(data)].shape[0]
        tIIe = data[~self.podmienka(data) & self.dosledok(data)].shape[0]
        tn =data[~self.podmienka(data) & ~self.dosledok(data)].shape[0]
        
        print(64*'═')
        print('║{:<20}║{:<20}|{:<20}║'.format('', 'Prediction positive', 'Prediction negative'))
        print(64*'═')
        print('║{:<20}║{:<20.2f}|{:<20.2f}║'.format('Condition positive', tp, tIIe))
        print(64*'═')
        print('║{:<20}║{:<20.2f}|{:<20.2f}║'.format('Condition negative', tIe, tn))
        print(64*'═')
        
        print("Accuracy: ", (tp + tn) / total if total != 0 else 0)
        print("Precission: ",  tp / (tp + tIe) if tp + tIe != 0 else 0)
        print("Recall: ", tp / (tp + tIIe) if tp + tIIe != 0 else 0)

Vrátili sme sa k pravidlám, ktoré sme v predchádzajúcej časti nemali dobre riešené.
Uvedomujúc si dôležitosť pozitívnej klasifikácie, tentokrát sme zvolili taký postup, že sme vytvárali podľa trénovacieho súboru také pravidlá, ktoré by maximalizovali detekciu pozitívnej triedy. 

Náš postup bol pritom taký, že sme začali s jedným pravidlom, ktoré vždy uhádlo pozitívnu triedu a na toto sme následne v snahe zvýšiť presnosť a znížiť chybu I. stupňa postupne nabaľovali ďalšie a ďalšie pravidlá, ktoré platili pre pozitívnu triedu.

Pomohli sme si pritom zisteniami z prieskumnej analýzy.

Začali sme  číselnými atribútmi.

In [49]:
pravidlo = Pravidlo('all', lambda a: ((a.T4U <= 1.6) & 
                                      (a.TT4 <= 240) & 
                                      (a.T3 <= 4.1) & 
                                      (a.TSH <= 2.5) & 
                                      (a.FTI <= 205)), lambda a: a.test == 'discordant')

print('---------------------------------------------\n')
print('Trénovací súbor: ')
pravidlo.over(train_data)

---------------------------------------------

Trénovací súbor: 
════════════════════════════════════════════════════════════════
║                    ║Prediction positive |Prediction negative ║
════════════════════════════════════════════════════════════════
║Condition positive  ║45.00               |0.00                ║
════════════════════════════════════════════════════════════════
║Condition negative  ║2486.00             |269.00              ║
════════════════════════════════════════════════════════════════
Accuracy:  0.11214285714285714
Precission:  0.017779533781114184
Recall:  1.0


Potom sme pridali aj kategorické.

In [52]:
pravidlo = Pravidlo('all', lambda a: ((a['on antithyroid medication'] == 'f') &
                                      (a['psych'] == 'f') &(a['sick'] == 'f') &
                                      (a['lithium'] == 'f') &
                                      (a['goitre'] == 'f') & 
                                      (a['tumor'] == 'f') &
                                      (a['I131 treatment'] == 'f') & 
                                      (a['thyroid surgery'] == 'f') & 
                                      (a['query on thyroxine'] == 'f') & 
                                      (a.T4U <= 1.6) & 
                                      (a.TT4 <= 240) & 
                                      (a.T3 <= 4.1) & 
                                      (a.TSH <= 2.5) &
                                      (a.FTI <= 205)), lambda a: a.test == 'discordant')

print('---------------------------------------------\n')
print('Trénovací súbor: ')
pravidlo.over(train_data)


---------------------------------------------

Trénovací súbor: 
════════════════════════════════════════════════════════════════
║                    ║Prediction positive |Prediction negative ║
════════════════════════════════════════════════════════════════
║Condition positive  ║43.00               |2.00                ║
════════════════════════════════════════════════════════════════
║Condition negative  ║2035.00             |720.00              ║
════════════════════════════════════════════════════════════════
Accuracy:  0.2725
Precission:  0.020692974013474495
Recall:  0.9555555555555556


Chybu 1. stupňa sme znížili približne o 25 percent. Pokúsili sa zapojiť ešte aj dolné ohraničenia pri číselných atribútoch.

In [53]:
pravidlo = Pravidlo('all', lambda a: ((a['on antithyroid medication'] == 'f') & 
                                      (a['psych'] == 'f') &
                                      (a['sick'] == 'f') &
                                      (a['lithium'] == 'f') &
                                      (a['goitre'] == 'f') & 
                                      (a['tumor'] == 'f') &
                                      (a['I131 treatment'] == 'f') & 
                                      (a['thyroid surgery'] == 'f') & 
                                      (a['query on thyroxine'] == 'f') & 
                                      (a.T4U >= 0.75) & 
                                      (a.TT4 >= 54) & 
                                      (a.T3 >= 1.2) & 
                                      (a.TSH >= -3.86) &  
                                      (a.FTI >= 58) &
                                      (a.T4U <= 1.6) & 
                                      (a.TT4 <= 240) & 
                                      (a.T3 <= 4.1) & 
                                      (a.TSH <= 2.5) & 
                                      (a.FTI <= 205)), 
                    lambda a: a.test == 'discordant')

print('---------------------------------------------\n')
print('Trénovací súbor: ')
pravidlo.over(train_data)


---------------------------------------------

Trénovací súbor: 
════════════════════════════════════════════════════════════════
║                    ║Prediction positive |Prediction negative ║
════════════════════════════════════════════════════════════════
║Condition positive  ║43.00               |2.00                ║
════════════════════════════════════════════════════════════════
║Condition negative  ║1791.00             |964.00              ║
════════════════════════════════════════════════════════════════
Accuracy:  0.35964285714285715
Precission:  0.023446019629225736
Recall:  0.9555555555555556


Takýmto spôsobom sa nám postupne podarilo znížiť chybu 1. stupňa približne na 66 percent. Viac atribútov už ale zapojiť do pravidiel nemôžeme a chyba 1. stupňa stále ostáva priveľká, preto musíme začať znižovať hranice testovania číselných atribútov. 
Začneme u tých, pri ktorých sme zistili možnú koreláciu s klasifikovaným atribútom.

In [54]:
pravidlo = Pravidlo('all', lambda a: ((a['on antithyroid medication'] == 'f') & 
                                      (a['psych'] == 'f') &
                                      (a['sick'] == 'f') &
                                      (a['lithium'] == 'f') &
                                      (a['goitre'] == 'f') & 
                                      (a['tumor'] == 'f') &
                                      (a['I131 treatment'] == 'f') & 
                                      (a['thyroid surgery'] == 'f') & 
                                      (a['query on thyroxine'] == 'f') & 
                                      (a['on thyroxine'] == 'f') & 
                                      (a.T4U >= 0.75) & 
                                      (a.TT4 >= 124) & 
                                      (a.T3 >= 1.2) & 
                                      (a.TSH >= -3.86) &  
                                      (a.FTI >= 150) &
                                      (a.T4U <= 1.6) & 
                                      (a.TT4 <= 220) & 
                                      (a.T3 <= 3.0) & 
                                      (a.TSH <= 2.5) & 
                                      (a.FTI <= 205)), 
                    lambda a: a.test == 'discordant')

print('---------------------------------------------\n')
print('Trénovací súbor: ')
pravidlo.over(train_data)


---------------------------------------------

Trénovací súbor: 
════════════════════════════════════════════════════════════════
║                    ║Prediction positive |Prediction negative ║
════════════════════════════════════════════════════════════════
║Condition positive  ║38.00               |7.00                ║
════════════════════════════════════════════════════════════════
║Condition negative  ║36.00               |2719.00             ║
════════════════════════════════════════════════════════════════
Accuracy:  0.9846428571428572
Precission:  0.5135135135135135
Recall:  0.8444444444444444


Toto sú už pravidlá, pri ktorých sa uspokojíme s ich výsledkom. Majú výbornú úroveň predikcie pre negatívnu triedu, pričom dobrá úroveň je aj pri predikovaní pozitívnej triedy. Chyba 1. bola znížená na uspokojujúcu uroveň. Znamená síce, že každé druhé predikovanie pozistívnej triedy je chybné, ale vzhľadom na predchádzajúce výsledky je to značný pokrok. Vyskúšame si teda teraz naše pravidlá na testovacom súbore.

In [55]:
pravidlo = Pravidlo('all', lambda a: ((a['on antithyroid medication'] == 'f') & 
                                      (a['psych'] == 'f') &
                                      (a['sick'] == 'f') &
                                      (a['lithium'] == 'f') &
                                      (a['goitre'] == 'f') & 
                                      (a['tumor'] == 'f') &
                                      (a['I131 treatment'] == 'f') & 
                                      (a['thyroid surgery'] == 'f') & 
                                      (a['query on thyroxine'] == 'f') & 
                                      (a['on thyroxine'] == 'f') & 
                                      (a.T4U >= 0.75) & 
                                      (a.TT4 >= 124) & 
                                      (a.T3 >= 1.2) & 
                                      (a.TSH >= -3.86) &  
                                      (a.FTI >= 150) &
                                      (a.T4U <= 1.6) & 
                                      (a.TT4 <= 220) & 
                                      (a.T3 <= 3.0) & 
                                      (a.TSH <= 2.5) & 
                                      (a.FTI <= 205)), 
                    lambda a: a.test == 'discordant')

print('---------------------------------------------\n')
print('Testovací súbor: ')
pravidlo.over(test_data)


---------------------------------------------

Testovací súbor: 
════════════════════════════════════════════════════════════════
║                    ║Prediction positive |Prediction negative ║
════════════════════════════════════════════════════════════════
║Condition positive  ║3.00                |10.00               ║
════════════════════════════════════════════════════════════════
║Condition negative  ║9.00                |950.00              ║
════════════════════════════════════════════════════════════════
Accuracy:  0.9804526748971193
Precission:  0.25
Recall:  0.23076923076923078


Pravidlá sme očividne pretrénovali. Chyby 1. aj 2. stupňa voči úspešnosti pozitívnej predikcie výrazne stúpli na úkor úspešnosti predikcie pozitívnej triedy. 

Porovnali sme si teda naše pravidlá s výsledkami stromu. Je vidieť, že strom je úspešnejší ako na trénovacom, tak aj na testovacom súbore, kde je rozdiel ešte poznateľnejší. My sme s našími pravidlami dosiahli síce menšiu chybu 1. stupnňa, ale v kontexte významu klasifikovaného atribútu je táto chyba menej podstatnejšia, ako chyba 1. stupňa.

In [57]:
print('---------------------------------------------\n')
print('Trénovací súbor: ')
print('\n***********************************************')
print('Naše pravidlá: ')
pravidlo.over(train_data)
print('\n***********************************************')
print('Strom: ')
tree_classifier_final_prediction = tree_classifier_final.predict(tree_train_data)
pravdivost(tree_train_target, tree_classifier_final_prediction)
print('\n══════════════════════════════════════════════════════════════════════════════════════════\n')
print('Testovací súbor: ')
print('\n***********************************************')
print('Naše pravidlá: ')
pravidlo.over(test_data)
print('\n***********************************************')
print('Strom: ')
tree_classifier_final_prediction = tree_classifier_final.predict(tree_test_data)
pravdivost(tree_test_target, tree_classifier_final_prediction)

---------------------------------------------

Trénovací súbor: 

***********************************************
Naše pravidlá: 
════════════════════════════════════════════════════════════════
║                    ║Prediction positive |Prediction negative ║
════════════════════════════════════════════════════════════════
║Condition positive  ║38.00               |7.00                ║
════════════════════════════════════════════════════════════════
║Condition negative  ║36.00               |2719.00             ║
════════════════════════════════════════════════════════════════
Accuracy:  0.9846428571428572
Precission:  0.5135135135135135
Recall:  0.8444444444444444

***********************************************
Strom: 
════════════════════════════════════════════════════════════════
║                    ║Prediction positive |Prediction negative ║
════════════════════════════════════════════════════════════════
║Condition positive  ║44.00               |1.00                ║
════════

Na týchto tabuľkách je aj vidieť, že strom bol na trénovacích dátach perfektne natrénovaný na klasifikáciu pozitívnej triedy a na elimináciu chyby 2. stupňa. 